In [ ]:
import sys

from active_critic.model_src.transformer import ModelSetup, TransformerModel, generate_square_subsequent_mask
from active_critic.model_src.base_transformer import DebugTEL
import torch as th
from active_critic.utils.pytorch_utils import calcMSE

In [ ]:
ms = ModelSetup()
seq_len = 3
ntoken = 3
batch_size = 2
d_output = 4

ms.d_output = d_output
ms.nhead = 2
ms.d_hid = 10
ms.d_model = 10
ms.nlayers = 1
ms.seq_len = seq_len
ms.dropout = 0
ms.ntoken = 1
ms.lr = None
ms.device = 'cpu'
device = 'cpu'


In [ ]:
def make_seq_encoding_data(batch_size, seq_len, ntoken, d_out, device = 'cuda'):
    inpt_seq = th.ones([batch_size,seq_len,ntoken], dtype=th.float, device=device)
    outpt_seq = th.ones([batch_size,seq_len,d_out], dtype=th.float, device=device)
    outpt_seq[:,::2] = 0
    return inpt_seq, outpt_seq

In [ ]:
tm = TransformerModel(model_setup=ms)
mask = generate_square_subsequent_mask(3)
mask = mask.unsqueeze(0).repeat([2, 1, 1])
mask[0] = 0
mask

In [14]:
def pull_tens_to_front(src, i):
    if i > 0:
        src[i, :, :-i] = src[i,:,i:]
        src[i, :, -i:] = -1

    return src

def pull_tens_to_front_sparse(src, i):
    pulled = src[i,:,i:]
    max_pulled = pulled.max(dim=-2).values == 1
    max_pulled = max_pulled.unsqueeze(-2).repeat([1, 1, src.shape[2] - i, 1])
    if i > 0:
        src[i, :, :-i] = max_pulled
        src[i, :, -i:] = -1
    else:
        src[i, :] = max_pulled

    return src

def repeat_along_seq_td(src):
    src = src.repeat([src.shape[1], 1, 1]).reshape([src.shape[1], src.shape[0], src.shape[1], src.shape[2]])
    return src

def repeat_along_seq(src, seq_len):
    src = src.repeat([1, seq_len, 1])
    return src

def generate_square_subsequent_mask_dense(seq_len, rewards):
    obsv = th.rand([2,3,4], requires_grad=True)


def make_dense_seq_encoding_data(actions, obsv, rewards):
    actions = repeat_along_seq_td(actions)
    obsv = repeat_along_seq_td(obsv)
    rewards = repeat_along_seq_td(rewards)
    for i in range(len(obsv)):
        obsv[i] = obsv[i,:,i].unsqueeze(1)

        actions = pull_tens_to_front(actions, i)
        rewards = pull_tens_to_front_sparse(rewards, i)
    return actions.reshape([-1, actions.shape[-2], actions.shape[-1]]), obsv.reshape([-1, obsv.shape[-2], obsv.shape[-1]]), rewards.reshape([-1, rewards.shape[-2], rewards.shape[-1]])

def generate_partial_observed_mask(reward, nheads):
    device = reward.device
    inv_result_mask = reward.squeeze() == -1
    result_mask = ~inv_result_mask
    args = th.argwhere(inv_result_mask)
    restructured_args = args.repeat([1, reward.shape[1]]).reshape([-1, 2])
    exp_ind = th.arange(reward.shape[1], device=device).repeat(args.shape[0])
    full_ind = th.cat((restructured_args[:, :1], exp_ind.unsqueeze(1), restructured_args[:, 1:]), dim=-1)
    attention_mask = th.zeros([reward.shape[0], reward.shape[1], reward.shape[1]], device=device)
    attention_mask[tuple(full_ind.T)] = -float('inf')
    attention_mask = attention_mask.repeat([1,1,nheads]).reshape([2*attention_mask.shape[0], attention_mask.shape[1], attention_mask.shape[2]])
    return attention_mask, result_mask

obsv = th.rand([2,3,4], requires_grad=True)
act = th.rand([2,3,2])
rewards = th.rand([2,3,1])

rewards[0,0,0] = 1
a, o, r = make_dense_seq_encoding_data(actions=act, obsv=obsv, rewards=rewards)
attention_mask, result_mask = generate_partial_observed_mask(r, ms.nhead)
o_req = o.detach()
o_req.requires_grad = True
print(f'o_req: {o_req.shape}')
print(f'attention_mask: {attention_mask.shape}')
result = tm.forward(src=o_req, mask=attention_mask)
result[result_mask].mean().backward()
o_req.grad

o_req: torch.Size([6, 3, 4])
attention_mask: torch.Size([12, 3, 3])
src: torch.Size([6, 3, 10])
mask: torch.Size([12, 3, 3])


tensor([[[ 0.0289, -0.0063,  0.0144, -0.0289],
         [ 0.0194,  0.0019,  0.0216, -0.0212],
         [ 0.0169,  0.0015,  0.0225, -0.0183]],

        [[ 0.0124, -0.0052,  0.0108, -0.0006],
         [ 0.0115, -0.0010,  0.0121, -0.0034],
         [ 0.0112, -0.0003,  0.0115, -0.0053]],

        [[ 0.0133,  0.0013,  0.0081, -0.0046],
         [ 0.0131,  0.0015,  0.0087, -0.0050],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0174,  0.0020,  0.0126,  0.0090],
         [ 0.0190,  0.0018,  0.0105,  0.0027],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0123, -0.0005,  0.0078, -0.0144],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0080, -0.0123,  0.0179,  0.0036],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]])

In [ ]:
obsv = th.rand([2,1,4], requires_grad=True)
rep_obsv = repeat_along_seq(src=obsv, seq_len=3)

In [ ]:
rewards[0,0,0] = 1
a, o, r = make_dense_seq_encoding_data(actions=act, obsv=obsv, rewards=rewards)

In [ ]:
attention_mask, result_mask = generate_partial_observed_mask(r)

In [ ]:
o_req = o.detach()
o_req.requires_grad = True

In [ ]:
result = tm.forward(src=o_req, mask=attention_mask)
result[result_mask].mean().backward()

In [ ]:
o_req.grad

In [ ]:
args = th.argwhere(r.squeeze() == -1)

In [ ]:
args

In [ ]:
restructured_args = args.repeat([1, 3]).reshape([18, 2])

In [ ]:
exp_ind = th.arange(3).repeat(6)

In [ ]:
full_ind = th.cat((restructured_args[:, :1], exp_ind.unsqueeze(1), restructured_args[:, 1:]), dim=-1)

In [ ]:
full_ind.shape

In [ ]:
mask = th.zeros([r.shape[0], r.shape[1], r.shape[1]])
mask[tuple(full_ind.T)] = -float('inf')

In [ ]:
mask

In [ ]:
exp_args = th.cat((args[:, :1], exp_ind))

In [ ]:
mask.shape

In [ ]:
mask

In [ ]:
r

In [ ]:
obsv

In [ ]:
obsv

In [ ]:
obsv

In [ ]:
def make_partial_sequence(seq):
    

In [ ]:
result = tm.forward(src=obsv, mask=mask)
loss = ((result[:, -2])**2).mean()
loss.backward()


In [ ]:
obsv.grad

In [ ]:
result

In [ ]:
def make_mask_data(batch_size, seq_len, ntoken, device = 'cuda'):
    mask = generate_square_subsequent_mask(seq_len).to(device)
    inpt_seq = th.ones([batch_size,seq_len,ntoken], dtype=th.float, device=device)
    inpt_seq[0,-1,0] = 0
    outpt_seq = th.ones_like(inpt_seq)
    outpt_seq[0] = 0
    return inpt_seq, outpt_seq, mask

In [ ]:
inpt_seq, outpt_seq = make_seq_encoding_data(batch_size, seq_len, ntoken, d_out = d_output, device=device)

In [ ]:
def make_no_conflict_part_data(batch_size, seq_len, obs_dim, d_out, device='cpu'):
    inpt = th.ones([batch_size, seq_len, obs_dim])
    inpt[0, 1:] = 2
    actions = th.arange(seq_len)
    actions = actions.reshape([1,-1,1]).repeat([batch_size, 1, d_out])
    actions[0] += 1

    res, actions = make_part_observed(inpt, actions)
    return res, actions
    

In [ ]:
def make_part_observed(inpt, actions):
    #batch, seq, dim
    inpt = inpt.unsqueeze(1).repeat([1,inpt.shape[1], 1, 1])
    inpt = inpt.permute([0,3,1,2])
    res = th.triu(inpt)
    res = res.permute([0,2,3,1]).reshape([-1,seq_len, obs_dim])
    
    rep_actions = actions.repeat([1,seq_len,1]).reshape([-1,seq_len, d_output])
    return res, rep_actions

In [ ]:
def make_conflict_part_data(batch_size, seq_len, obs_dim, d_out, device='cpu'):
    inpt = th.ones([batch_size, seq_len, obs_dim])
    inpt[0, :1] = 2
    actions = th.arange(seq_len)
    actions = actions.reshape([1,-1,1]).repeat([batch_size, 1, d_out])
    actions[0, :1] += 1

    res, actions = make_part_observed(inpt, actions)
    rew = th.ones([batch_size * seq_len, seq_len], device = device)
    
    return res, actions, rew

In [ ]:
def make_conflict_part_data_neg(batch_size, seq_len, obs_dim, d_out, device='cpu'):
    inpt = th.ones([batch_size, seq_len, obs_dim])
    inpt[0, -1:] = 2
    actions = th.arange(seq_len)
    actions = actions.reshape([1,-1,1]).repeat([batch_size, 1, d_out])
    actions[0, -1:] += 1

    res, actions = make_part_observed(inpt, actions)
    rew = th.ones([batch_size * seq_len, seq_len], device = device)
    scale = th.arange(seq_len).reshape([1,-1]).repeat([rew.shape[0], 1])
    rew = rew * scale / (seq_len - 1)
    rew[:,-1:] = 0
    
    res_copy = res.clone()
    res[:seq_len] = res_copy[seq_len:]
    res[seq_len:] = res_copy[:seq_len]
    
    return res, actions, rew

In [ ]:
def make_sequence(obs, acts):
    result = th.cat((obs, acts), dim=-1)
    return result

In [ ]:
obs_dim = 3


In [ ]:
obs_nc, act_nc = make_no_conflict_part_data(batch_size, seq_len, obs_dim, d_output, device='cpu')
obs_c, act_c, rew_c_pos = make_conflict_part_data(batch_size, seq_len, obs_dim, d_output, device='cpu')
obs_c_n, act_c_n, rew_c_neg = make_conflict_part_data_neg(batch_size, seq_len, obs_dim, d_output, device='cpu')

In [ ]:
rew_c_neg

In [ ]:
seq_n = make_sequence(obs_c_n,act_c_n)
seq_p = make_sequence(obs_c,act_c)

In [ ]:
seq = th.cat((seq_n, seq_p), dim=0)
rew = th.cat((rew_c_pos, rew_c_neg), dim=0)

In [ ]:
ms = ModelSetup()
seq_len = 3
ntoken = 3 + 4
batch_size = 2
d_output = 1

ms.d_output = d_output
ms.nhead = 1
ms.d_hid = 10
ms.d_model = 10
ms.nlayers = 1
ms.seq_len = seq_len
ms.dropout = 0
ms.ntoken = 1
ms.lr = None
ms.device = 'cpu'
ms.optimizer_class = th.optim.AdamW
ms.optimizer_kwargs = {}
ms.model_class:TransformerModel = TransformerModel
device = 'cpu'


In [ ]:
inpt = seq

model = TransformerModel(model_setup=ms).to(device)
with th.no_grad():
    answer = model.forward(inpt)
optimizer = th.optim.Adam(params=model.parameters(), lr=1e-3)
loss = 0
for i in range(3000):
    result = model.forward(inpt)
    loss = calcMSE(result, rew)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
loss

In [ ]:
inpt[0]

In [ ]:
answer = model.forward(inpt[5].unsqueeze(0))

In [ ]:
answer

In [ ]:
rew[5]

In [ ]:
inpt[9]

In [ ]:
result, attention = model.forward(res_c[0].unsqueeze(0), return_attention=True)

In [ ]:
result

In [ ]:
attention

In [ ]:
result, attention = model.forward(res[4].unsqueeze(0), return_attention=True)

In [ ]:
attention

In [ ]:
res[4]

In [ ]:
mask = generate_square_subsequent_mask(seq_len).to('cuda')

In [ ]:
inpt_seq = th.ones([2,seq_len,1], dtype=th.float, device='cuda')
inpt_seq[0,-1,0] = 0

outpt_seq = th.ones_like(inpt_seq)
outpt_seq[0] = 0

In [ ]:
inpt_seq, outpt_seq, mask = make_mask_data(batch_size=batch_size, seq_len=seq_len, ntoken=ntoken)


In [ ]:
model = TransformerModel(model_setup=ms).to('cuda')
with th.no_grad():
    model.forward(inpt_seq)
optimizer = th.optim.Adam(params=model.parameters(), lr=1e-3)
loss = 0
for i in range(1000):
    result = model.forward(inpt_seq, mask=None)
    loss = calcMSE(result, outpt_seq)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
loss

In [ ]:
from ActiveCritic.model_src.transformer import CriticTransformer

In [ ]:
ms = ModelSetup()
seq_len = 6
d_output = 1
ms.d_output = d_output
ms.nhead = 1
ms.d_hid = 10
ms.d_model = 10
ms.nlayers = 2
ms.seq_len = seq_len
ms.dropout = 0
ms.ntoken = 1
ms.lr = None
ms.device = 'cuda'
ms.optimizer_class = th.optim.AdamW
ms.optimizer_kwargs = {}
ms.d_result = 1
ms.model_class:TransformerModel = CriticTransformer

In [ ]:
inpt_seq = th.ones([2,seq_len,4], dtype=th.float, device='cuda')
inpt_seq[0,-1,0] = 0

outpt_seq = th.ones([2,1], dtype=th.float, device='cuda')
outpt_seq[0] = 0

In [ ]:
model = CriticTransformer(model_setup=ms).to('cuda')
with th.no_grad():
    a = model.forward(inpt_seq)

In [ ]:
a.shape

In [ ]:
model = CriticTransformer(model_setup=ms).to('cuda')
with th.no_grad():
    model.forward(inpt_seq)
optimizer = th.optim.Adam(params=model.parameters(), lr=1e-3)
loss = 0
for i in range(2000):
    result = model.forward(inpt_seq)
    loss = calcMSE(result, outpt_seq)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
loss